## Tensorflow - DNN

### 1. Import Relevant Package

In [ ]:
import functools

import numpy as np
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
train_file_path = "cardio_normal_train.csv"
test_file_path = "cardio_normal_test.csv"

### 2. Load the Data

In [ ]:
# label column
LABEL_COLUMN = 'cardio'
LABELS = [0, 1]

In [ ]:
def get_dataset(file_path):
    """
    Set up tensorflow dataset format
    """
    dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=12,
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True)
    return dataset

# load train and test data
raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [ ]:
# test
examples, labels = next(iter(raw_train_data))
print("EXAMPLES: \n", examples, "\n")
print("LABELS: \n", labels)

### 3. Preprocessing of the data

##### The datasets we import have already been processed in the data cleaning and EDA part, so as for this preprocessing procedure, we don't need to deal with the data.

In [ ]:
feature_columns = []
feature_layer_inputs = {}

for header in ['age', 'cholesterol', 'gluc', 'height', 'weight', 'ap_hi', 'ap_lo', 'BMI', 'gender', 'smoke', 'alco', 'active']:
    feature_columns.append(tf.feature_column.numeric_column(header))
    feature_layer_inputs[header] = tf.keras.Input(shape=(1,), name=header)

### 4. Set up a DNN

##### -- Input layer

In [ ]:
preprocessing_layer = tf.keras.layers.DenseFeatures(feature_columns)

##### --  Start from `preprocessing_layer` to set up `tf.keras.Sequential`.

In [ ]:
# DNN - h(g(f(x)))
model = tf.keras.Sequential([
    preprocessing_layer,  # input layer
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),   # output layer
])

In [ ]:
model.compile(
    loss='binary_crossentropy',       
    optimizer='adam',
    metrics=['accuracy'])

### 5. Train, Measure & Prediction

##### -- Now we can train the model.

In [ ]:
train_data = raw_train_data.shuffle(500)
test_data = raw_test_data

In [ ]:
model.fit(train_data, epochs=20)

In [ ]:
model.fit?

In [ ]:
model.summary()

##### -- Test the accuracy on the test dataset `test_data` .

In [ ]:
test_loss, test_accuracy = model.evaluate(test_data)

print()
print(f'Test Loss {test_loss}, Test Accuracy {test_accuracy}')

##### -- Prediction (using `tf.keras.Model.predict` )

In [ ]:
predictions = model.predict(test_data)
predictions[:10]

In [ ]:
list(test_data)[0][1]

In [ ]:
# show some results
for prediction, cardio in zip(predictions[:10], list(test_data)[0][1][:10]):
    is_cardio = "cardio" if bool(cardio) else "non-cardio"
    print(f"predict: {prediction[0]} | real: {is_cardio}")